In [1]:
import warnings
warnings.filterwarnings("ignore")

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import LabelEncoder, OneHotEncoder, StandardScaler, MinMaxScaler
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.metrics import accuracy_score, f1_score, roc_auc_score, roc_curve, auc, mean_squared_error, classification_report
from sklearn import  model_selection
from sklearn import ensemble
from sklearn import svm

from sklearn.model_selection import GridSearchCV
from vecstack import stacking, StackingTransformer
from xgboost import XGBClassifier
from IPython.display import Image
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression
from Preprocessing import Preprocessing
from sklearn.model_selection import train_test_split

In [2]:
preprocessing_db = Preprocessing()
preprocessing_db.drop_columns()

In [3]:
preprocessing_db = Preprocessing()
# 결측치 제거
preprocessing_db.drop_columns()
# 연속형 데이터 대체
preprocessing_db.numerical_columns_replace()
# 범주형 데이터 대체ㅐ
preprocessing_db.category_columns_replace()

In [4]:
df = preprocessing_db.get_df()

In [5]:
target = preprocessing_db.get_target_df()

In [6]:
category_columns = [
    'Car_Owned', 'Bike_Owned', 'Active_Loan', 
    'House_Own', 'Homephone_Tag', 'Workphone_Working', 
    'Cleint_City_Rating', 'Application_Process_Day', 'Client_Permanent_Match_Tag', 
    'Client_Contact_Work_Tag']

In [7]:
category_columns_object = [
    'Client_Education', 'Client_Marital_Status', 
    'Client_Gender', 'Loan_Contract_Type', 'Client_Housing_Type', 
    'Client_Occupation']

In [8]:
numerical_columns=[
    'Child_Count', 'Credit_Amount', 'Loan_Annuity', 
    'Population_Region_Relative', 'Age_Days', 'Employed_Days', 
    'Registration_Days', 'ID_Days', 'Client_Family_Members',
    'Score_Source_2', 'Phone_Change', 'Credit_Bureau', 'Client_Income']

In [ ]:
x_train_df = pd.DataFrame(x_train_transformed)

In [ ]:
x_train_df.to_csv('./x_train_df.csv', index=False)

In [ ]:
y_train_df = pd.DataFrame(y_train)

In [27]:
pd.set_option('display.max_rows', None)
pay_df = pd.read_csv('./nbfi_vehicle_loan_repayment_dataset/Train_Dataset.csv', encoding='utf-8', engine='python')
na_values = ['$', '#VALUE!', '##', 'XNA', '@', '#', 'x', '&']

    
['Accompany_Client', 'Client_Gender', 'Type_Organization']
pay_df['Accompany_Client'].value_counts()


Alone       97409
Relative    15748
Partner      4516
Kids         1334
Others        987
Group         104
##             12
Name: Accompany_Client, dtype: int64

In [ ]:
y_train_df.to_csv('./y_train_df.csv', index=False)

In [ ]:
df['Default'].value_counts()

In [ ]:
target_df = preprocessing_db.get_target_df()

In [ ]:
target_df

In [ ]:
na_values = ['$', '#VALUE!', '##', 'XNA', '@', '#', 'x', '&']
columns = ['Client_Income', 'Credit_Amount', 'Loan_Annuity', 'Score_Source_3', 'Population_Region_Relative', 'Age_Days', 'Employed_Days', 'Registration_Days', 'ID_Days']
pay_df = pd.read_csv('./nbfi_vehicle_loan_repayment_dataset/Train_Dataset.csv', na_values=na_values, encoding='utf-8', engine='python')

for column in columns:
    pay_df[column] = pd.to_numeric(pay_df[column], errors='coerce')

In [ ]:
x_test = pd.read_csv('./nbfi_vehicle_loan_repayment_dataset/x_test.csv')
x_train = pd.read_csv('./nbfi_vehicle_loan_repayment_dataset/x_train_over.csv')
y_train = pd.read_csv('./nbfi_vehicle_loan_repayment_dataset/y_train_over.csv')
y_test = pd.read_csv('./nbfi_vehicle_loan_repayment_dataset/y_test.csv')

In [ ]:
df.shape

In [ ]:
df = pd.concat([x_test, x_train])
y_df = pd.concat([y_train, y_test])
df.to_csv('./data.csv', index=False)
y_df.to_csv('./y_data.csv', index=False)

In [ ]:
pd.set_option('display.max_rows', None)

In [ ]:
# pd.get_dummies(df, columns=['Accompany_Client', 'Application_Process_Hour'], ax=1)

In [ ]:
df.Cleint_City_Rating.value_counts()

In [ ]:
df_null_sum = df.isna().sum()
column = df_null_sum[df_null_sum>0].index
test = pd.get_dummies(df, columns=column)

In [ ]:
pd.set_option('display.max_rows', None)

In [ ]:
test.isna().sum()

In [ ]:
df.isna().sum()

In [ ]:
df = pd.read_csv('./nbfi_vehicle_loan_repayment_dataset/Train_Dataset.csv', na_values=['$', '#VALUE!', '##', 'XNA', '@', '#', 'x', '&'], encoding='utf-8', engine='python')

In [ ]:
# 범주형인데 1, 2~~~
category_value_number = ['Car_Owned', 'Bike_Owned',
                        'Active_Loan', 'House_Own', 'Child_Count',
                        'Own_House_Age', 'Homephone_Tag', 'Workphone_Working', 
                        'Client_Family_Members', 'Cleint_City_Rating', 'Application_Process_Day', 'Application_Process_Hour',
                        'Credit_Bureau']

In [ ]:
# 연속형
numerical_value = ['Client_Income', 'Credit_Amount', 'Loan_Annuity', 
                   'Population_Region_Relative', 'Age_Days', 'Employed_Days',
                   'Registration_Days', 'ID_Days', 'Score_Source_1' ,
                   'Score_Source_2', 'Score_Source_3', 'Social_Circle_Default', 'Phone_Change']

In [ ]:
# 범주형인데 str
category_value_object = df.select_dtypes(include='object').columns

## 결측치가 만 건 미만인 컬럼명
- Accompany_Client,고객이 대출을 신청할 때 고객과 동행한 사람 : Drop
- Client_Income_Type,고객 소득 유형 : Drop
- Client_Education,고객이 달성한 최고 교육 수준 : Random
- Client_Marital_Status,"고객의 결혼 상태(D- 이혼, S- 미혼, M- 기혼, W- 사별)" : Random
- Client_Gender,고객의 성별 : Random
- Loan_Contract_Type,"대출 유형(CL- 현금대출, RL- 리볼빙대출)" : Random
- Client_Housing_Type,고객 주택 상황 : Random

In [ ]:
columns = {}
category_value_object = pay_df.select_dtypes(include='object').columns

for column in category_value_object:
    if pay_df[column].isna().sum() < 10000:
        continue
    columns[column] = pay_df[column].isna().sum()

columns

In [ ]:
pd.DataFrame({"Count" : df.isna().sum(), "Percentage" : df.isna().sum() / len(df) * 100})

# 주제 : 차량 대출 상환가능성 예측

In [ ]:
aa = ['Own_House_Age', 'Client_Occupation', 'Type_Organization', 'Score_Source_1', 'Score_Source_3', 'Social_Circle_Default', 'Credit_Bureau']

## 결측치가 만건 이상인 컬럼명
- Own_House_Age,고객 소유 주택의 나이(년) : drop
- Client_Occupation,고객 직업 유형 : Nojob :: DROP
- Type_Organization,클라이언트가 근무하는 조직 유형 : drop
- Score_Source_1,다른 출처에서 얻은 점수. 정규화된 점수입니다. : drop
- Score_Source_3,다른 소스에서 가져온 점수입니다. 정규화된 점수입니다. : drop
- Social_Circle_Default,지난 60일 동안 대출 상환을 불이행한 고객의 친구/가족 수 : drop
- Credit_Bureau, 1년 내 90일 이상 장기연체 등 신용위험이 발생할 가능성을 수치화하여 제공하는 지표 : 평균

- ID,고객 대출 신청 ID
- Client_Income,고객 소득() : DROP
- Car_Owned,다른 차량에 대한 대출을 신청하기 전에 고객이 소유한 모든 차량(0은 아니요, 1은 그렇지 않음을 의미)
- Bike_Owned,고객이 소유한 모든 자전거(0은 아니요, 1은 그렇지 않음을 의미)
- Active_Loan,대출 신청 당시 진행 중인 다른 대출이 있는지 여부(0은 아니요, 1은 그렇지 않음을 의미)
- House_Own,고객이 소유한 주택 수(0은 아니요, 1은 그 외의 주택)
- Child_Count,고객이 보유한 자녀 수
- Credit_Amount,대출의 신용 금액()
- Loan_Annuity,대출 연금()
- Accompany_Client,고객이 대출을 신청할 때 고객과 동행한 사람
- Client_Income_Type,고객 소득 유형
- Client_Education,고객이 달성한 최고 교육 수준
- Client_Marital_Status,"고객의 결혼 상태(D- 이혼, S- 미혼, M- 기혼, W- 사별)"
- Client_Gender,고객의 성별
- Loan_Contract_Type,"대출 유형(CL- 현금대출, RL- 리볼빙대출)"
- Client_Housing_Type,고객 주택 상황
- Population_Region_Relative,고객이 거주하고 있는 지역의 상대적 인구수. 값이 높을수록 고객이 인구가 많은 지역에 거주하고 있음을 의미합니다.
- Age_Days,신청서 제출 시점의 고객 나이
- Employed_Days,"신청 전, 고객이 수입을 시작한 일수"
- Registration_Days,"대출 신청일 전, 고객이 등록을 변경한 일수"
- ID_Days,"대출 신청 며칠 전, 고객이 대출을 신청한 신분증을 변경했습니다."
- Own_House_Age,고객 소유 주택의 나이(년)
- Mobile_Tag,고객이 제공한 휴대폰 번호(1은 예, 0은 아니오를 의미) : drop
- Homephone_Tag,고객이 제공한 집전화 번호(1은 예, 0은 아니오를 의미)
- Workphone_Working,직장 전화 번호로 연락 가능했는지(1은 예, 0은 아니오)
- Client_Occupation,고객 직업 유형
- Client_Family_Members,고객이 보유한 가족 구성원 수
- Cleint_City_Rating,고객 도시 등급. 3은 최고, 2는 좋음, 1은 평균을 나타냅니다.
- Application_Process_Day,고객이 대출을 신청한 요일(0-일,1-월,2-화,3-수,4-목,5-금,6-토)"
- Application_Process_Hour,고객이 대출을 신청한 요일 시간 : DROP
- Client_Permanent_Match_Tag,고객 연락처 주소가 영구 주소와 일치하지 않는 경우 표시.
- Client_Contact_Work_Tag,고객 직장 주소가 연락처 주소와 일치하지 않는 경우 표시.
- Type_Organization,클라이언트가 근무하는 조직 유형
- Score_Source_1,다른 출처에서 얻은 점수. 정규화된 점수입니다.
- Score_Source_2,다른 출처에서 얻은 점수입니다. 정규화된 점수입니다.
- Score_Source_3,다른 소스에서 가져온 점수입니다. 정규화된 점수입니다.
- Social_Circle_Default,지난 60일 동안 대출 상환을 불이행한 고객의 친구/가족 수
- Phone_Change,"대출 신청 며칠 전에 고객이 휴대폰을 변경했는지"
- Credit_Bureau,작년 총 조회 건수
- Default,1은 고객이 대출 상환을 불이행했음을 의미하고 0은 그렇지 않음을 의미합니다.

In [ ]:
# pay_df.Client_Gender = count_encoder.fit_transform(pay_df.Client_Gender)

pay_df.Client_Gender = label_encoder.fit_transform(pay_df.Client_Gender)

In [ ]:
pay_df.Client_Gender

In [ ]:
pay_df.Accompany_Client.value_counts()
# pay_df.Accompany_Client.isna().sum()

In [ ]:
pay_df.Client_Gender.value_counts()

In [ ]:
pay_df.Client_Gender = pay_df.Client_Gender.apply(lambda x : 0 if x == 2 else x)

In [ ]:
pay_df.pivot_table(index=y_target, columns=['Client_Gender'], aggfunc=['mean', 'sum'])

In [ ]:
train_data, test_data, train_label, test_label = model_selection.train_test_split(df, target,
                                                                                 test_size=0.3,
                                                                                 random_state=42)

In [ ]:
# OverSampling

In [ ]:
numerical_columns

In [ ]:
category_columns.extend(category_columns_object)
category_columns

In [ ]:
numeric_transformer = StandardScaler() # cf) RobustScaler
categorical_transformer = OneHotEncoder(categories='auto', handle_unknown='ignore') 

preprocessor = ColumnTransformer(
    transformers=[ # List of (name, transformer, column(s))
        ('num', numeric_transformer, numerical_columns),
        ('cat', categorical_transformer, category_columns)])

In [ ]:
preprocessor_pipe = Pipeline(steps=[('preprocessor', preprocessor)]) # preprocessing-only

In [ ]:
preprocessor_pipe.fit(train_data)

In [ ]:
x_train_transformed = preprocessor_pipe.transform(train_data)
x_test_transformed = preprocessor_pipe.transform(test_data)

In [ ]:
estimators = [ 
    ('ExtraTrees', ensemble.ExtraTreesClassifier(random_state = 0, n_jobs = -1, n_estimators = 100, max_depth = 3)),
    ('RandomForest', ensemble.RandomForestClassifier(random_state = 0, n_jobs = -1, n_estimators = 100, max_depth = 3)),
    ('DecisionTree', DecisionTreeClassifier()),
]

In [ ]:
stack = StackingTransformer(estimators, 
                            regression = False, 
                            metric = accuracy_score,
                            n_folds = 2, stratified = True, shuffle = True, 
                            random_state = 42, verbose = 2) 

In [ ]:
# fit을 통해 분류함
stack = stack.fit(x_train_transformed, train_label)

In [ ]:
# transform을 통해 학습시킴
S_train = stack.transform(x_train_transformed)
# transform을 통해 학습시킴
S_test = stack.transform(x_test_transformed)

In [ ]:
# XGBClassification의 hyper-parameter
params = {
    'learning_rate' : [1, 0.1, 0.01, 0.001],
    'n_estimators' : [100, 1000],
    'max_depth' : [3, 4, 5],
    'eval_metric' : ['mlogloss'],
    'n_jobs' : [-1],
    'seed' : [0],
}

In [ ]:
# GridSearch를 통해 HPO 찾기
grid = GridSearchCV(
    XGBClassifier(),
    params,
    refit=True, verbose=1
)
y_pred = grid.fit(S_train, train_label)
y_pred.best_params_

In [ ]:
# HPO 적용
model = XGBClassifier(**y_pred.best_params_) 
# 투표를 통해 얻은 S_train과 y_train을 다시 학습시킴
model = model.fit(S_train, y_train)

In [ ]:
fpr, tpr, _ = roc_curve(y_true=y_test, y_score=model.predict_proba(S_test)[:,1])
roc_auc = auc(fpr, tpr) # AUC 면적의 값 (수치)
plt.figure(figsize=(9, 9))
plt.plot(fpr, tpr, color='darkorange', lw=2, label='ROC curve (area = %0.2f)' % roc_auc)
plt.plot([0, 1], [0, 1], color='navy', lw=2, linestyle='--')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.legend(loc="lower right")
plt.title("ROC curve")
plt.show()

In [ ]:
predictions = model.predict(S_test)
print(classification_report(y_test, predictions))

In [ ]:
from sklearn.svm import SVC
svc = SVC()

In [ ]:
params = {
    'C' : [1, 10, 100, 1000],
    'kernel' : ['rbf'],
    'gamma' : [0.1, 0.01, 0.001],
    'max_iter' : [3, 4, 5],
}


# GridSearch를 통해 HPO 찾기
grid = GridSearchCV(
    svc,
    params,
    refit=True, verbose=1
)
y_pred = grid.fit(x, y_train)
y_pred.best_params_

In [ ]:
scv = SVC(**y_pred.best_params_)

In [ ]:
scv.fit(S_train, y_train)

In [ ]:
pridict_y = scv.predict(x_test)

In [ ]:
print(classification_report(y_test, pridict_y))

In [ ]:
fpr, tpr, _ = roc_curve(y_true=y_test, y_score=scv.predict_proba(S_test)[:,1])
roc_auc = auc(fpr, tpr) # AUC 면적의 값 (수치)
plt.figure(figsize=(9, 9))
plt.plot(fpr, tpr, color='darkorange', lw=2, label='ROC curve (area = %0.2f)' % roc_auc)
plt.plot([0, 1], [0, 1], color='navy', lw=2, linestyle='--')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.legend(loc="lower right")
plt.title("ROC curve")
plt.show()